In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # data visualization
import os 

for dirname, _, filenames in os.walk('/kaggle/input'): # fetch dataset
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


### Acquire data:

In [2]:
def read_data():
    train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
    print("Train data imported successfully!!")
    print("-"*50)
    test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
    print("Test data imported successfully!!")
    return train_data , test_data
train_data , test_data = read_data()

Train data imported successfully!!
--------------------------------------------------
Test data imported successfully!!


In [3]:
# see the data structure
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#### Filling Blanks and Missed Data:

In [4]:
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
test_data.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

##### It's important to fill Age, Embarked, Fare features: 

In [6]:
train_data['Embarked'] = train_data.Embarked.fillna(train_data.Embarked.dropna().max()) # 用最大值fillNA
test_data['Fare'] = test_data.Fare.fillna(test_data.Fare.dropna().mean()) # 用平均值fillNA

##### we will guess the age from Pclass and Sex:

In [7]:
guess_ages = np.zeros((2,3))
guess_ages

array([[0., 0., 0.],
       [0., 0., 0.]])

Now we iterate over Sex (0 or 1) and Pclass (1, 2, 3) to calculate guessed values of Age for the six combinations.

In [8]:
combine = [train_data , test_data]

# Converting Sex categories (male and female) to 0 and 1: 
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

# Filling missed age feature:

for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & \
                                  (dataset['Pclass'] == j+1)]['Age'].dropna()
            age_guess = guess_df.median() # 根据Sex和Pclass填补Age为其对应的中位数

            # Convert random age float to nearest .5 age
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
            
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]

    dataset['Age'] = dataset['Age'].astype(int)

train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",1,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",0,35,0,0,373450,8.0500,NaN,S


In [9]:
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [10]:
test_data.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64

##### There are no important missed data anymore!!!

### Modeling:

In [11]:
# ========================================================= #
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
# ========================================================= #
from sklearn.tree import DecisionTreeClassifier
# ========================================================= #
from colorama import Fore

In [12]:
# First we will select the features that we will use:

features = ["Pclass" , "Sex" , "Age" , "SibSp" , "Parch" , "Fare" , "Embarked"]

# Categorical to indicator variables: 类别值0-1编码
X_train = pd.get_dummies(train_data[features])
Y_train = train_data["Survived"]
X_test = pd.get_dummies(test_data[features])

X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,3,0,22,1,0,7.2500,0,0,1
1,1,1,38,1,0,71.2833,1,0,0
2,3,1,26,0,0,7.9250,0,0,1
3,1,1,35,1,0,53.1000,0,0,1
4,3,0,35,0,0,8.0500,0,0,1


In [13]:
def print_scores(model ,X_train , Y_train,predictions , cv_splites=10):
    # Fore 不同颜色区分结果
    # 通过mean accuracy和cross validation及test score看出是欠拟合、正常还是过拟合
    # 欠拟合 = train val test acc都低，正常 = train val test acc差不多都高， 过拟合 = train 远高于 val test
    print(Fore.BLUE , "The mean accuracy score of the train data is %.5f" % model.score(X_train, Y_train))
    CV_scores = cross_val_score(model, X_train, Y_train, cv=cv_splites)
    print(Fore.BLACK ,"The individual cross-validation scores are: \n",CV_scores)
    print(Fore.BLACK ,"The minimum cross-validation score is %.3f" % min(CV_scores))
    print(Fore.BLACK ,"The maximum cross-validation score is %.3f" % max(CV_scores))
    print(Fore.YELLOW ,"The mean  cross-validation   score is %.5f ± %0.2f" % (CV_scores.mean(), CV_scores.std() * 2))
    #print(Fore.RED ,"The test (i.e. leaderboard)  score is %.5f (this score is unknown)" % accuracy_score(true_solution["Survived"],predictions))

In [14]:
# 通过max_depth, max_features调整模型复杂度，从而调整模型的拟合程度
model = DecisionTreeClassifier(max_depth=3 , max_features=4 ,random_state=7)
model.fit(X_train, Y_train)
predictions = model.predict(X_test)
print_scores(model, X_train, Y_train, predictions)

 The mean accuracy score of the train data is 0.82379
 The individual cross-validation scores are: 
 [0.82222222 0.84269663 0.7752809  0.85393258 0.84269663 0.79775281
 0.80898876 0.78651685 0.85393258 0.83146067]
 The minimum cross-validation score is 0.775
 The maximum cross-validation score is 0.854
 The mean  cross-validation   score is 0.82155 ± 0.05


In [15]:
from sklearn.ensemble import RandomForestClassifier
# 随机森林
model = RandomForestClassifier(n_estimators= 80 ,max_depth=5 , max_features=8 ,min_samples_split=3 ,random_state=7)
model.fit(X_train, Y_train)
predictions = model.predict(X_test)
print_scores(model, X_train, Y_train, predictions)

 The mean accuracy score of the train data is 0.85859
 The individual cross-validation scores are: 
 [0.76666667 0.85393258 0.75280899 0.91011236 0.88764045 0.80898876
 0.80898876 0.78651685 0.87640449 0.84269663]
 The minimum cross-validation score is 0.753
 The maximum cross-validation score is 0.910
 The mean  cross-validation   score is 0.82948 ± 0.10


#### Conclusions:

- Titanic Dataset hasn't enough training examples, And this leads to Overfitting.

- For other problems with big datasets you can control Overfitting and Underfitting by:  

    -1- tradeoff between simple and complex models to find the sweet spot.  
    -2- cross validation score very important to measure Overfitting and Underfitting  
    -3- use grid search to find the best hyperparameters.  